In [212]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
from random import *
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
os.chdir('C:\\Users\\Kepa\\Documents\\Github2\\Hoffman_Leinwand_COMP755_Project')

In [213]:
Demand_Matrix = pd.read_csv('Demand_Matrix.csv', header= None)
Demand_Matrix = Demand_Matrix.values
Time_Matrix = pd.read_csv('Time_Matrix.csv', header = None)
Time_Matrix = Time_Matrix.values
Time_Matrix[Time_Matrix == -0] = np.Inf
OTime_dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv')
ODemand_dataframe = pandas.read_csv('Demand_Data3_Total_CV_2017_with_zeros.csv')

PU_DO_list = Time_dataframe['PU_DO']
PU_DO_list = np.unique(PU_DO_list)
PU_DO_list = PU_DO_list.tolist()

PULoc_list = np.unique(Demand_dataframe['PULocationID'])
PULoc_list = PULoc_list.tolist() 

#Remember to change:
chosen_Time_list = 1262

dictionary = {'100': 0, '107': 1, '113': 2, '114': 3, '12': 4, '125': 5, '13': 6, '137': 7, '140': 8, '141': 9, '142': 10, '143': 11, '144': 12, '148': 13, '151': 14, '158': 15, '161': 16, '162': 17, '163': 18, '164': 19, '170': 20, '186': 21, '209': 22, '211': 23, '224': 24, '229': 25, '230': 26, '231': 27, '232': 28, '233': 29, '234': 30, '236': 31, '237': 32, '238': 33, '239': 34, '24': 35, '246': 36, '249': 37, '261': 38, '262': 39, '263': 40, '4': 41, '43': 42, '45': 43, '48': 44, '50': 45, '68': 46, '79': 47, '87': 48, '88': 49, '90': 50}

In [214]:
print(OTime_dataframe)

         Unnamed: 0  OLS_CV_Prediction  OLS_Prediction  OLS_fitted_value  \
0                 0         406.913122       -1.000000        407.939954   
1                 1         296.322699       -1.000000        296.614283   
2                 2         376.331127       -1.000000        376.436955   
3                 3         387.360999       -1.000000        386.823059   
4                 4         421.656756       -1.000000        423.522370   
5                 5         261.749575       -1.000000        260.728148   
6                 6         363.038591       -1.000000        362.397935   
7                 7         257.651050       -1.000000        257.602779   
8                 8         324.504269       -1.000000        324.761492   
9                 9         295.462329       -1.000000        295.705884   
10               10         246.562657       -1.000000        246.351286   
11               11         320.373957       -1.000000        320.900264   
12          

In [215]:
print(Time_Matrix)

[[373.3523865         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]
 ...
 [        inf         inf         inf ...         inf 157.7098694
          inf]
 [        inf         inf         inf ... 229.2384033         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]]


In [258]:
correct_demand_decision = 0
correct_time_decision = 0
correct_discount_decision = 0
for i in range(len(Demand_Matrix)):
    current_loc = str(PULoc_list[i])
    #print('current_loc', current_loc)
    neighbors = []
    for j in range(len(PU_DO_list)):
        #For each location, get its neighbors
        current_PU_DO = PU_DO_list[j]
        [current_loc_candidate,destination]  = current_PU_DO.split('_')
        if current_loc_candidate != current_loc:
            pass
        else:
            neighbors.append(destination)
    #print(neighbors)
    
    #Find the estiamted and true demand/time for each neighbor
    Estimated_Demand = []
    True_Demand = []
    Estimated_Time = []
    True_Time = []
    for j in range(len(neighbors)):
        current_neighbor = neighbors[j]
        #print('current neighbor', current_neighbor)
        
        #True Demand
        dataframe = ODemand_dataframe
        dataframe = dataframe[dataframe['PULocationID'] == int(current_neighbor)]
        dataframe = dataframe[dataframe['Month'] > 10]
        dataframe = dataframe['passenger_count']
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        dataset = np.reshape(dataset, (len(dataset),1))

        #print('This is the True deamnd:', test2Actual)
        True_Demand.append(dataset[chosen_Time_list])
        
        #Estimated Demand
        #print('Estimated Demand', Demand_Matrix[int(dictionary[current_neighbor])])
        Estimated_Demand.append( PULoc_list.index(int(current_neighbor)))
        
        
        #True Time
        dataframe = OTime_dataframe
        PULoc = str(current_loc) + '_' + str(current_neighbor)
        dataframe = dataframe[dataframe['PU_DO'] == PULoc]
        dataframe = dataframe[dataframe['True_Month'] > 10]
        dataframe = dataframe['time_delta']
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        dataset = np.reshape(dataset, (len(dataset),1))
        #print('True Time', dataset[chosen_Time_list] )
        tru_tim = dataset[chosen_Time_list]
        
        #If true time = 0, set to minimum:
        if  dataset[chosen_Time_list]==0:
            tru_tim = min( dataset[dataset > 0] )
            
        True_Time.append(tru_tim)
        
        #Estimated Time
        row = dictionary[current_loc]
        col = dictionary[current_neighbor]
        #print('Estimated Time:',Time_Matrix[row, col])
        Estimated_Time.append(Time_Matrix[row, col])
    
    #Add the current location
    #True Demand
    dataframe = ODemand_dataframe
    dataframe = dataframe[dataframe['PULocationID'] == int(current_loc)]
    dataframe = dataframe[dataframe['Month'] > 10]
    dataframe = dataframe['passenger_count']
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    dataset = np.reshape(dataset, (len(dataset),1))
    True_Demand.append(dataset[chosen_Time_list])
    True_Demand = np.asarray(True_Demand)
    True_Demand = np.reshape(True_Demand, (len(True_Demand),))

    #Estimated Demand
    Estimated_Demand.append( PULoc_list.index(int(current_loc)))
    Estimated_Demand = np.asarray(Estimated_Demand)

        
    #True Time
    True_Time.append(0)
    True_Time = np.asarray(True_Time)
    
    #Estimated Time
    Estimated_Time.append(0)
    Estimated_Time = np.asarray(Estimated_Time)
    
        
        
    #Percentage correct guess which neighbor has the best demand
    actual_choice = np.argmax(True_Demand)
    estimated_choice = np.argmax(Estimated_Demand)
    if actual_choice == estimated_choice:
        correct_demand_decision = correct_demand_decision+1
        
        
    
    #Percentage correct guess which neighbor has the best time (infront cuz 0)
    actual_choice = np.argmin(True_Time)
    estimated_choice = np.argmin(Estimated_Time)       
    if actual_choice == estimated_choice:
        correct_time_decision = correct_time_decision+1

        
    #DIscounting FUnction
    Estimated_Discount =  Estimated_Demand - Estimated_Time/3600 * Estimated_Demand
    print('Estimated Discount', Estimated_Discount)
    print('True demand shape:', True_Demand.shape)
    print('True Time shape:', True_Time.shape)
    True_Discount = True_Demand - True_Time/3600 * True_Demand
    print('True Discount', True_Discount)
    
    #Percentage correct guess which neighbor has the best time (infront cuz 0)
    actual_choice = np.argmax(True_Discount)
    estimated_choice = np.argmax(Estimated_Discount)       
    if actual_choice == estimated_choice:
        correct_discount_decision = correct_discount_decision+1
        
    

        
        
        
correct_demand_decision = correct_demand_decision/  len(Demand_Matrix)  
correct_time_decision = correct_time_decision/  len(Demand_Matrix)  
correct_discount_decision = correct_discount_decision/len(Demand_Matrix)

    

Estimated Discount [34.07294333 36.21095126  8.33005249  0.        ]
True demand shape: (4,)
True Time shape: (4,)
True Discount [ 38.89166667  10.91333333 275.58277778  29.        ]
Estimated Discount [ 1.96044989 47.21258519 10.91561044  1.        ]
True demand shape: (4,)
True Time shape: (4,)
True Discount [137.30111111 171.42666667  36.96916667  10.        ]
Estimated Discount [ 0.97549904 34.11517254 44.39125325  2.        ]
True demand shape: (4,)
True Time shape: (4,)
True Discount [array([9.1472225], dtype=float32) array([171.51953], dtype=float32)
 array([156.28111], dtype=float32) 139.0]
Estimated Discount [23.88847245  3.76295646  3.        ]
True demand shape: (3,)
True Time shape: (3,)
True Discount [array([153.77222], dtype=float32) array([296.84], dtype=float32) 42.0]
Estimated Discount [18.93899831 22.79584906 25.60979407 37.59290029 39.23537524 37.92907549
 40.32773704  2.84154589  4.        ]
True demand shape: (9,)
True Time shape: (9,)
True Discount [array([447.626

Estimated Discount [18.929611   24.21772224 24.41177107 32.55830157 39.0309491   3.55319536
  5.29174393 29.        ]
True demand shape: (8,)
True Time shape: (8,)
True Discount [array([456.49777], dtype=float32) array([484.8336], dtype=float32)
 array([338.6062], dtype=float32) array([290.48077], dtype=float32)
 array([643.05304], dtype=float32) array([307.32144], dtype=float32)
 array([304.36554], dtype=float32) 417.0]
Estimated Discount [11.49155167 23.69202805 28.38657271 29.55705322 37.85416527 30.        ]
True demand shape: (6,)
True Time shape: (6,)
True Discount [array([250.30344], dtype=float32) array([482.8145], dtype=float32)
 array([345.3625], dtype=float32) array([363.02567], dtype=float32)
 array([426.84134], dtype=float32) 248.0]
Estimated Discount [12.66861798 16.14504868 24.14375702 24.60312249 27.59886703 37.02982076
 31.        ]
True demand shape: (7,)
True Time shape: (7,)
True Discount [array([281.6128], dtype=float32) array([129.86111], dtype=float32)
 array([49

In [259]:
#Percent of the Time we correctly guessed which area would be the Hottest accofdint to LSTM
print('Demand correct', correct_demand_decision)
print('Time correct', correct_time_decision)
print('Discount correct', correct_discount_decision)

Demand correct 0.29411764705882354
Time correct 1.0
Discount correct 0.29411764705882354


In [203]:
print(np.asarray(Estimated_Time)/ 3600)

[0.07072811 0.06272894 0.05643051 0.05643051]


In [220]:
print(Time_Matrix)

[[373.3523865         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]
 ...
 [        inf         inf         inf ...         inf 157.7098694
          inf]
 [        inf         inf         inf ... 229.2384033         inf
          inf]
 [        inf         inf         inf ...         inf         inf
          inf]]
